In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [2]:
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [4]:
# !wget -O Data-collisions.csv https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv 

--2020-09-07 03:06:53--  https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv
Resolving s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)... 67.228.254.196
Connecting to s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73917638 (70M) [text/csv]
Saving to: ‘Data-collisions.csv’

Data-collisions.csv 100%[===================>]  70.49M  20.1MB/s    in 3.6s    

2020-09-07 03:06:57 (19.3 MB/s) - ‘Data-collisions.csv’ saved [73917638/73917638]



In [3]:
collision_df = pd.read_csv('Data-collisions.csv')

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
collision_cdf=collision_df[['SEVERITYCODE','COLLISIONTYPE','PERSONCOUNT','PEDCOUNT','PEDCYLCOUNT','VEHCOUNT','INATTENTIONIND','UNDERINFL',
                            'SDOT_COLCODE','JUNCTIONTYPE','LIGHTCOND','ROADCOND','WEATHER','HITPARKEDCAR','SPEEDING','OBJECTID','INTKEY','PEDROWNOTGRNT']]

collision_cdf_T=collision_cdf.copy()

#Convert Categories to Numbers
collision_cdf_T["LIGHTCOND"] = collision_cdf_T["LIGHTCOND"].astype('category')
collision_cdf_T["ROADCOND"] = collision_cdf_T["ROADCOND"].astype('category')
collision_cdf_T["WEATHER"] = collision_cdf_T["WEATHER"].astype('category')
collision_cdf_T["INATTENTIONIND"] = collision_cdf_T["INATTENTIONIND"].astype('category')
collision_cdf_T["UNDERINFL"] = collision_cdf_T["UNDERINFL"].astype('category')
collision_cdf_T["HITPARKEDCAR"] = collision_cdf_T["HITPARKEDCAR"].astype('category')
collision_cdf_T["SPEEDING"] = collision_cdf_T["SPEEDING"].astype('category')
collision_cdf_T["COLLISIONTYPE"] = collision_cdf_T["COLLISIONTYPE"].astype('category')
collision_cdf_T["JUNCTIONTYPE"] = collision_cdf_T["JUNCTIONTYPE"].astype('category')
collision_cdf_T["PEDROWNOTGRNT"] = collision_cdf_T["PEDROWNOTGRNT"].astype('category')

collision_cdf_T["LIGHTCOND_N"] = collision_cdf_T["LIGHTCOND"].cat.codes
collision_cdf_T["ROADCOND_N"] = collision_cdf_T["ROADCOND"].cat.codes
collision_cdf_T["WEATHER_N"] = collision_cdf_T["WEATHER"].cat.codes
collision_cdf_T["INATTENTIONIND_N"] = collision_cdf_T["INATTENTIONIND"].cat.codes
collision_cdf_T["UNDERINFL_N"] = collision_cdf_T["UNDERINFL"].cat.codes
collision_cdf_T["HITPARKEDCAR_N"] = collision_cdf_T["HITPARKEDCAR"].cat.codes
collision_cdf_T["SPEEDING_N"] = collision_cdf_T["SPEEDING"].cat.codes
collision_cdf_T["COLLISIONTYPE_N"] = collision_cdf_T["COLLISIONTYPE"].cat.codes
collision_cdf_T["JUNCTIONTYPE_N"] = collision_cdf_T["JUNCTIONTYPE"].cat.codes
collision_cdf_T["PEDROWNOTGRNT_N"] = collision_cdf_T["PEDROWNOTGRNT"].cat.codes

#Drop ORiginal Field
collision_cdf_T.drop(['INATTENTIONIND','LIGHTCOND','ROADCOND','WEATHER','UNDERINFL','SPEEDING','HITPARKEDCAR','COLLISIONTYPE','JUNCTIONTYPE','PEDROWNOTGRNT'], axis=1, inplace=True)

#Normalize Data Field INTKEY and OBJECTID
collision_cdf_T["INTKEY"] = collision_cdf_T["INTKEY"] / collision_cdf_T["INTKEY"].max()
collision_cdf_T['INTKEY'] = collision_cdf_T['INTKEY'].replace(np.nan, 0)
collision_cdf_T["OBJECTID"] = collision_cdf_T["OBJECTID"] / collision_cdf_T["OBJECTID"].max()
collision_cdf_T['OBJECTID'] = collision_cdf_T['OBJECTID'].replace(np.nan, 0)

#Normalize Data FieldSEVERITYCODE
collision_cdf_T["SEVERITYCODE"] = collision_cdf_T["SEVERITYCODE"] -1

In [5]:
#Divide the data into Train and Test DataSets
msk = np.random.rand(len(collision_cdf_T)) < 0.8
train = collision_cdf_T[msk]
test = collision_cdf_T[~msk]

In [6]:
#Using Recursive Feature Elimination for selecting the features
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

Predictor_Matrix=['PERSONCOUNT','PEDCOUNT','PEDCYLCOUNT','VEHCOUNT','SDOT_COLCODE',
                  'OBJECTID','INTKEY','LIGHTCOND_N','ROADCOND_N','WEATHER_N',
                  'INATTENTIONIND_N','UNDERINFL_N','HITPARKEDCAR_N','SPEEDING_N',
                  'COLLISIONTYPE_N','JUNCTIONTYPE_N','PEDROWNOTGRNT_N']
                  

train_x = np.asanyarray(train[Predictor_Matrix])
train_y = np.asanyarray(train['SEVERITYCODE'])

rfe = RFE(LogisticRegression(random_state=None, solver='liblinear'), 20)
rfe = rfe.fit(train_x, np.ravel(train_y,order='C'))
print(rfe.support_)
print(rfe.ranking_)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [7]:
import statsmodels.api as sm
logit_model=sm.Logit(train_y,train_x)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.523185
         Iterations 7
                          Results: Logit
Model:              Logit            Pseudo R-squared: 0.143      
Dependent Variable: y                AIC:              162967.5430
Date:               2020-09-07 18:09 BIC:              163136.7911
No. Observations:   155713           Log-Likelihood:   -81467.    
Df Model:           16               LL-Null:          -95101.    
Df Residuals:       155696           LLR p-value:      0.0000     
Converged:          1.0000           Scale:            1.0000     
No. Iterations:     7.0000                                        
--------------------------------------------------------------------
           Coef.    Std.Err.      z       P>|z|     [0.025    0.975]
--------------------------------------------------------------------
x1         0.2155     0.0054    40.1422   0.0000    0.2050    0.2261
x2         2.8216     0.0487    57.9403   0.0

In [12]:
train_x = np.asanyarray(train[Predictor_Matrix])
train_y = np.asanyarray(train['SEVERITYCODE'])

logreg = LogisticRegression(random_state=None, solver='liblinear')
logreg.fit(train_x, train_y)

print ('Coefficients: ', logreg.coef_)
print ('Intercept: ',logreg.intercept_)
print ('Classifications: ', logreg.classes_)

test_x = np.asanyarray(test[Predictor_Matrix])
test_y = np.asanyarray(test[['SEVERITYCODE']])
pred_y = logreg.predict(test_x)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(test_x, test_y)))

from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(test_y, pred_y)
print('Confusion Matrix')
print(confusion_matrix)

from sklearn.metrics import classification_report
print(classification_report(test_y, pred_y))

Coefficients:  [[ 0.22809642  3.14902851  2.53388605  0.32139592  0.04252421  0.09961493
   0.90720787 -0.01449907  0.03281563 -0.10574626  0.30349503  0.04092285
  -1.50226352  0.49506366 -0.07339613 -0.14287007]]
Intercept:  [-1.22858147]
Classifications:  [0 1]
Accuracy of logistic regression classifier on test set: 0.75
Confusion Matrix
[[26299   932]
 [ 8682  2880]]
              precision    recall  f1-score   support

           0       0.75      0.97      0.85     27231
           1       0.76      0.25      0.37     11562

   micro avg       0.75      0.75      0.75     38793
   macro avg       0.75      0.61      0.61     38793
weighted avg       0.75      0.75      0.71     38793

